In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
# Данный код монтирует гугл диск к директории /gdrive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
train = pd.read_csv('/gdrive/My Drive/DATA/Stepik/orange_small_churn_train_data.csv')
train.shape

(18299, 232)

In [0]:
test = pd.read_csv('/gdrive/My Drive/DATA/Stepik/orange_small_churn_test_data.csv')
test.shape

(10000, 231)

In [0]:
from sklearn.preprocessing import LabelEncoder

def labelEncoder(data):
    #data = data.fillna(0)
    le = LabelEncoder()
    for m in range(190,230):
        #print(m)
        tmp = data.iloc[:,m].astype(str)
        le.fit(tmp)
        data.iloc[:,m] = le.transform(tmp)        
    return data

In [0]:
# проведем испытания XGBoost
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from scipy.stats import uniform, randint
from xgboost import XGBClassifier
import time

In [0]:
x_data_le = labelEncoder(train.iloc[:,:230])

In [0]:
x_train = x_data_le
y_train = train['labels']

In [0]:
clf = XGBClassifier()

In [0]:
param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

In [0]:
y_train_nan = y_train.drop([18298])
x_train_nan = x_train.drop([18298])
x_train_nan = x_train_nan.fillna(0)
y_train_nan = y_train_nan.fillna(0)

In [0]:
rs_clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=1, verbose=2, cv=2,
                            scoring='neg_log_loss', refit=False, random_state=42)
print("Randomized search..")
search_time_start = time.time()
rs_clf.fit(x_train_nan[x_train_nan.columns[1:231]], y_train_nan)
print("Randomized search time:", time.time() - search_time_start)

best_score = rs_clf.best_score_
best_params = rs_clf.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

Randomized search..
Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV] subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=800, min_child_weight=7.0, max_depth=10, learning_rate=3, gamma=1.0, colsample_bytree=0.6, colsample_bylevel=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.7s remaining:    0.0s


[CV]  subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=800, min_child_weight=7.0, max_depth=10, learning_rate=3, gamma=1.0, colsample_bytree=0.6, colsample_bylevel=0.8, total=  13.7s
[CV] subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=800, min_child_weight=7.0, max_depth=10, learning_rate=3, gamma=1.0, colsample_bytree=0.6, colsample_bylevel=0.8 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[CV]  subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=800, min_child_weight=7.0, max_depth=10, learning_rate=3, gamma=1.0, colsample_bytree=0.6, colsample_bylevel=0.8, total=  13.5s
[CV] subsample=0.7, silent=False, reg_lambda=0.1, n_estimators=300, min_child_weight=7.0, max_depth=6, learning_rate=0.1, gamma=0.5, colsample_bytree=0.7, colsample_bylevel=0.8 
[CV]  subsample=0.7, silent=False, reg_lambda=0.1, n_estimators=300, min_child_weight=7.0, max_depth=6, learning_rate=0.1, gamma=0.5, colsample_bytree=0.7, colsample_bylevel=0.8, total=  21.9s
[CV] subsample=0.7, silent=False, reg_lambda=0.1, n_estimators=300, min_child_weight=7.0, max_depth=6, learning_rate=0.1, gamma=0.5, colsample_bytree=0.7, colsample_bylevel=0.8 
[CV]  subsample=0.7, silent=False, reg_lambda=0.1, n_estimators=300, min_child_weight=7.0, max_depth=6, learning_rate=0.1, gamma=0.5, colsample_bytree=0.7, colsample_bylevel=0.8, total=  22.0s
[CV] subsample=0.6, silent=False, reg_lambda=10.0, n_estimators=80

...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[CV]  subsample=0.5, silent=False, reg_lambda=5.0, n_estimators=100, min_child_weight=1.0, max_depth=6, learning_rate=3, gamma=0.5, colsample_bytree=0.4, colsample_bylevel=0.9, total=   3.2s
[CV] subsample=1.0, silent=False, reg_lambda=0.1, n_estimators=600, min_child_weight=1.0, max_depth=15, learning_rate=0.001, gamma=0, colsample_bytree=0.7, colsample_bylevel=0.7 
[CV]  subsample=1.0, silent=False, reg_lambda=0.1, n_estimators=600, min_child_weight=1.0, max_depth=15, learning_rate=0.001, gamma=0, colsample_bytree=0.7, colsample_bylevel=0.7, total= 1.4min
[CV] subsample=1.0, silent=False, reg_lambda=0.1, n_estimators=600, min_child_weight=1.0, max_depth=15, learning_rate=0.001, gamma=0, colsample_bytree=0.7, colsample_bylevel=0.7 
[CV]  subsample=1.0, silent=False, reg_lambda=0.1, n_estimators=600, min_child_weight=1.0, max_depth=15, learning_rate=0.001, gamma=0, colsample_bytree=0.7, colsample_bylevel=0.7, total= 1.4min
[CV] subsample=1.0, silent=False, reg_lambda=1.0, n_estimators=

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:2295: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[CV]  subsample=1.0, silent=False, reg_lambda=100.0, n_estimators=100, min_child_weight=10.0, max_depth=20, learning_rate=3, gamma=0.25, colsample_bytree=0.4, colsample_bylevel=0.4, total=   1.4s
[CV] subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=700, min_child_weight=5.0, max_depth=6, learning_rate=0.1, gamma=0, colsample_bytree=1.0, colsample_bylevel=0.4 
[CV]  subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=700, min_child_weight=5.0, max_depth=6, learning_rate=0.1, gamma=0, colsample_bytree=1.0, colsample_bylevel=0.4, total=  34.8s
[CV] subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=700, min_child_weight=5.0, max_depth=6, learning_rate=0.1, gamma=0, colsample_bytree=1.0, colsample_bylevel=0.4 
[CV]  subsample=0.9, silent=False, reg_lambda=5.0, n_estimators=700, min_child_weight=5.0, max_depth=6, learning_rate=0.1, gamma=0, colsample_bytree=1.0, colsample_bylevel=0.4, total=  35.0s
Randomized search time: 2191.8018317222595
Best score: -0.248779387616

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 36.5min finished


In [0]:
clf_best = XGBClassifier(colsample_bylevel=0.7, colsample_bytree=0.4, gamma = 0.5, learning_rate=0.01, max_depth=15, min_child_weight=5.0, n_estimators=600,
                         reg_lambda=1.0, silent=False, subsample=1.0)

In [0]:
clf_best.fit(x_train_nan[x_train_nan.columns[1:231]], y_train_nan)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.5,
              learning_rate=0.01, max_delta_step=0, max_depth=15,
              min_child_weight=5.0, missing=None, n_estimators=600, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1.0, scale_pos_weight=1, seed=None,
              silent=False, subsample=1.0, verbosity=1)

In [0]:
x_data_test_le = labelEncoder(test.iloc[:,:230])

In [0]:
x_data_test_le = x_data_test_le.fillna(0)

In [0]:
pred_prob_1 = clf_best.predict(x_data_test_le[x_data_test_le.columns[1:]])

In [0]:
df = pd.DataFrame({'ID': x_data_test_le.index, 'result': pred_prob_1})
df.to_csv('answer11.csv', index = False)
from google.colab import files
files.download('answer11.csv')

In [0]:
pred_prob_2 = clf_best.predict_proba(x_data_test_le[x_data_test_le.columns[1:]]).transpose()

In [0]:
df = pd.DataFrame({'ID': x_data_test_le.index, 'result': pred_prob_2[1]})
df.to_csv('answer12.csv', index = False)
from google.colab import files
files.download('answer12.csv')

### Result 0.70218